In [ ]:
import pandas as pd
import matplotlib as plt
import numpy as np
import os
from bertopic.representation import KeyBERTInspired
from bertopic import BERTopic
from hdbscan import HDBSCAN
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', 10)
pd.set_option('display.max_rows', 50)
print(os.getcwd())

In [ ]:
df_cluster_refied = pd.read_csv('Cluster_Tweet_Without_Generic.csv')

In [ ]:
df_alt = pd.read_csv('Collected_Fake_Tweets_Still_Available.csv')
df_alt_report = pd.read_csv('AltNews Report Number.csv')
df_alt = pd.merge(df_alt, df_alt_report[['News_Number', 'Fact_Check_Link']])

In [ ]:
df_cluster_refied.Cluster_ID = df_cluster_refied.Cluster_ID.astype('int')
df_cluster_refied['Cluster_ID'] = 'Wild_' + df_cluster_refied['Cluster_ID'].astype('str')

In [ ]:
df_cluster_small = df_cluster_refied[['Cluster_ID', 'Tweet_ID', 'User_ID', 'Tweet_Text']]

In [ ]:
df_cluster_small.groupby('Cluster_ID').count()

In [ ]:
df_alt_small = df_alt[['News_Number', 'Tweet_ID', 'User_ID', 'Tweet_Text']].rename(columns = {'News_Number': 'Cluster_ID'})

In [ ]:
df_alt_small['Cluster_ID'] = 'Alt_'+df_alt_small['Cluster_ID'].astype('str')

In [ ]:
df_cluster_small['Camp_Type'] = 'WildCamp'
df_alt_small['Camp_Type'] = 'AltNews'

In [ ]:
df_all_small = pd.concat([df_cluster_small, df_alt_small])

In [ ]:
import re
def remove_twitter_mentions(text):
    return re.sub(r'@\w+', '', text)

def remove_links(text):
    return re.sub(r'http\S+', '', text)


In [ ]:
df_all_small['Refined_Text'] = df_all_small['Tweet_Text'].apply(remove_twitter_mentions)
df_all_small['Refined_Text'] = df_all_small['Refined_Text'].apply(remove_links)
df_all_small['Refined_Text'] = df_all_small['Refined_Text'].str.replace('\n', ' ')
df_all_small['Refined_Text'] = df_all_small['Refined_Text'].str.strip()

In [ ]:
df_all_small

In [ ]:
df_all_small_unique = df_all_small.groupby('Cluster_ID').apply(lambda x: x.sample(1)).reset_index(drop=True)

In [ ]:
df_all_small_unique.sample()

In [ ]:
docs_all = df_all_small['Refined_Text'].tolist()
docs_wild = df_all_small.loc[df_all_small.Camp_Type == 'WildCamp']['Refined_Text'].tolist()
docs_all_unique = df_all_small_unique['Refined_Text'].tolist()
docs_wild_unique = df_all_small_unique.loc[df_all_small_unique.Camp_Type == 'WildCamp']['Refined_Text'].tolist()

In [ ]:
hdbscan_model1 = HDBSCAN(min_cluster_size=4, metric='euclidean', 
                        cluster_selection_method='eom', prediction_data=True, min_samples=5)
representation_model1 = KeyBERTInspired()

hdbscan_model2 = HDBSCAN(min_cluster_size=4, metric='euclidean', 
                        cluster_selection_method='eom', prediction_data=True, min_samples=5)
representation_model2 = KeyBERTInspired()

hdbscan_model3 = HDBSCAN(min_cluster_size=2, metric='euclidean', 
                        cluster_selection_method='eom', prediction_data=True, min_samples=2)
representation_model3 = KeyBERTInspired()

hdbscan_model4 = HDBSCAN(min_cluster_size=2, metric='euclidean', 
                        cluster_selection_method='eom', prediction_data=True, min_samples=2)
representation_model4 = KeyBERTInspired()


In [ ]:
topic_model_all = BERTopic(representation_model=representation_model1, language="multilingual", hdbscan_model=hdbscan_model1)
topic_model_all_unique = BERTopic(representation_model=representation_model2, language="multilingual", hdbscan_model=hdbscan_model3)
topic_model_wild = BERTopic(representation_model=representation_model3, language="multilingual", hdbscan_model=hdbscan_model2)
topic_model_wild_unique = BERTopic(representation_model=representation_model4, language="multilingual", hdbscan_model=hdbscan_model4)

In [ ]:
topics_all, probs_all = topic_model_all.fit_transform(docs_all)
new_topics_all = topic_model_all.reduce_outliers(docs_all, topics_all)
topic_model_all.save("/disk/mnemo/users/randomuser/SBERT All Embedding/Group_Topic/Topic_All", serialization="pickle")

In [ ]:
topics_all_unique, probs_all_unique = topic_model_all_unique.fit_transform(docs_all_unique)
new_topics_all_unique = topic_model_all_unique.reduce_outliers(docs_all_unique, topics_all_unique)
topic_model_all_unique.save("/disk/mnemo/users/randomuser/SBERT All Embedding/Group_Topic/Topic_All_Unique", serialization="pickle")

In [ ]:
topics_wild, probs_wild = topic_model_wild.fit_transform(docs_wild)
new_topics_wild = topic_model_wild.reduce_outliers(docs_wild, topics_wild)
topic_model_wild.save("/disk/mnemo/users/randomuser/SBERT All Embedding/Group_Topic/Topic_Wild", serialization="pickle")

In [ ]:
topics_wild_unique, probs_wild_unique = topic_model_wild_unique.fit_transform(docs_wild_unique)
new_topics_wild_unique = topic_model_wild_unique.reduce_outliers(docs_wild_unique, topics_wild_unique)
topic_model_wild_unique.save("/disk/mnemo/users/randomuser/SBERT All Embedding/Group_Topic/Topic_Wild_Unique", serialization="pickle")

In [ ]:
topic_model_all_unique.get_topic_info().iloc[20:40]

In [ ]:
topic_model_wild_unique.get_topic_info().iloc[0:30]

In [ ]:
new_topics_wild_unique

In [ ]:
df_all_small_unique.loc[df_all_small_unique.Camp_Type == 'WildCamp']['Topic'] = new_topics_wild_unique

In [ ]:
df_wild_unique = df_all_small_unique.loc[df_all_small_unique.Camp_Type == 'WildCamp']

In [ ]:
df_wild_unique['Reduce_Topic'] = new_topics_wild_unique
df_wild_unique['Actual_Topic'] = topics_wild_unique

In [ ]:
df_wild_unique.groupby('Actual_Topic').count()

In [ ]:
topic_model_wild_unique.get_topic_info().iloc[100:200]

In [ ]:
print(1-2)

In [ ]:
topic_model_wild_unique = BERTopic.load("/disk/mnemo/users/randomuser/SBERT All Embedding/Group_Topic/Topic_Wild_Unique")

In [ ]:
topic_model_wild_unique.get_params()

In [ ]:
topic_model_wild_unique.generate_topic_labels()

In [ ]:


hdbscan_model5 = HDBSCAN(min_cluster_size=2, metric='euclidean', 
                        cluster_selection_method='eom', prediction_data=True, min_samples=2)
representation_model5 = KeyBERTInspired()

topic_model_wild_unique_fixed_topic = BERTopic(representation_model=representation_model5, language="multilingual", hdbscan_model=hdbscan_model5, nr_topics=100)

In [ ]:
topics_wild_unique_fixed, probs_wild_unique_fixed = topic_model_wild_unique_fixed_topic.fit_transform(docs_wild_unique)
new_topics_wild_unique_fixed = topic_model_wild_unique_fixed_topic.reduce_outliers(docs_wild_unique, topics_wild_unique_fixed)
topic_model_wild_unique_fixed_topic.save("/disk/mnemo/users/randomuser/SBERT All Embedding/Group_Topic/Topic_Wild_Unique_Fixed", serialization="pickle")

In [ ]:
df_com = pd.read_csv('Refined_Community_with_Political_Label.csv')

In [ ]:
topic_model_wild_unique.get_topic_info()

In [ ]:
df_wild_unique.groupby('Reduce_Topic').count()

In [ ]:
df_wild_unique

In [ ]:
df_com.User_ID.tolist()

In [ ]:
df_com.loc[df_com.Community_ID == 0].User_ID

In [ ]:
df_wild_unique.loc[df_wild_unique.Topic == 0]

In [ ]:
df_wild_unique[df_wild_unique.User_ID.isin(df_com.loc[df_com.Community_ID == 0].User_ID)].sample(20)

In [ ]:
df_cluster_particate = pd.read_csv('Cluster_Participate_Group.csv')

In [ ]:
df_cluster_particate

In [ ]:
top_20_topic = df_wild_unique[df_wild_unique.Cluster_ID.isin(df_cluster_particate.Cluster_ID)].groupby('Reduce_Topic').count().sort_values('Cluster_ID', ascending = False).iloc[0:20].reset_index()

In [ ]:
top_20_topic = df_wild_unique[df_wild_unique.Cluster_ID.isin(df_cluster_particate.Cluster_ID)].groupby('Actual_Topic').count()

In [ ]:
topic_model_wild_unique.get_topic_info().loc[topic_model_wild_unique.get_topic_info().Topic == 9625]

In [ ]:
top_20_topic 

In [ ]:
topic_model_wild_unique.get_topic_info()[topic_model_wild_unique.get_topic_info().Topic.isin(top_20_topic.Reduce_Topic)]

In [ ]:
df_wild_unique.loc[df_wild_unique.Topic == 0].Refined_Text.tolist()

In [ ]:
df_wild_unique[['Cluster_ID', 'Topic']]

In [ ]:
df_cluster_particate=df_cluster_particate.drop(columns = ['index'])

In [ ]:
df_cluster_particate = pd.merge(df_cluster_particate,df_wild_unique[['Cluster_ID', 'Topic']])

In [ ]:
print(len(df_cluster_particate.loc[df_cluster_particate.Community_ID == 0].groupby('Topic').count()))

In [ ]:
grp_li = [0,
18,
1,
17,
8,
6,
15,
29,
19,
13,
5,
4,
14,
42]

for i in grp_li:
    print(i)
    print(len(df_cluster_particate.loc[df_cluster_particate.Community_ID == i].groupby('Topic').count()))

In [ ]:
count_topic_size = df_wild_unique.groupby('Topic').count().reset_index()

In [ ]:
count_topic_size10=count_topic_size.loc[count_topic_size.Cluster_ID>=10]

In [ ]:
topic_model_wild_unique.get_topic_info().iloc[1:11].to_csv('Top-10-Topic.csv', index = False)

In [ ]:
count_topic_size10.Topic

In [ ]:
new_top10_campaign = df_wild_unique[df_wild_unique.Topic.isin(count_topic_size10.Topic)]

In [ ]:
df_cluster_particate_top10 = pd.merge(df_cluster_particate, new_top10_campaign[['Cluster_ID', 'Topic']])

In [ ]:
grp_li = [0,
18,
1,
17,
8,
6,
15,
29,
19,
13,
5,
4,
14,
42]

for i in grp_li:
    print(i)
    print(len(df_cluster_particate_top10.loc[df_cluster_particate_top10.Community_ID == i].groupby('Topic').count()))

In [ ]:
topic_model_wild_unique.get_topic_info().iloc[1:21].to_csv('Top_20_Topic.csv', index = False)

In [ ]:
topic_model_wild_unique.get_topic_info().to_csv('All_Topic_Wild_Unique.csv', index = False)

In [ ]:
topic_model_wild_unique.get_topic_info().iloc[10:20]

In [ ]:
df_cluster_particate.loc[df_cluster_particate.Community_ID == 0]

In [ ]:
pd.merge(df_wild_unique.loc[df_wild_unique.Topic == 10], df_cluster_particate.loc[df_cluster_particate.Community_ID == 0]).sample(20)

In [ ]:
pd.merge(df_wild_unique.loc[df_wild_unique.Topic == 10], df_cluster_particate.loc[df_cluster_particate.Community_ID == 17])

In [ ]:
topic_model_wild_unique.get_topic_info().iloc[1:21]

In [ ]:
df_wild_unique

In [ ]:
df_all_tweet_cluster = pd.read_csv('Cluster_Tweet_Without_Generic.csv')

In [ ]:
df_all_tweet_cluster.Cluster_ID = df_all_tweet_cluster.Cluster_ID.astype('int')

In [ ]:
df_wild_unique[['Cluster_ID', 'Actual_Topic', 'Reduce_Topic']]

In [ ]:
df_all_tweet_cluster['Cluster_ID']= 'Wild_' + df_all_tweet_cluster['Cluster_ID'].astype('str')

In [ ]:
df_all_tweet_cluster = pd.merge(df_all_tweet_cluster,df_wild_unique[['Cluster_ID', 'Actual_Topic', 'Reduce_Topic']])

In [ ]:
df_wild_unique

In [ ]:
for i in range(0,21):
    print(len(df_all_tweet_cluster.loc[df_all_tweet_cluster.Actual_Topic == i]))

In [ ]:
df_all_tweet_cluster.Tweet_Lang

In [ ]:
df_all_tweet_cluster.loc[df_all_tweet_cluster.Tweet_Lang == 'en'].groupby('Cluster_ID').count()

In [ ]:
df_wild_unique.loc[df_wild_unique.Tweet_Lang == 'en']

In [ ]:
df_wild_unique

In [ ]:
df_cluster_refied[df_cluster_refied.Tweet_Lang == 'en'].groupby('Cluster_ID').count().reset_index().Cluster_ID.tolist()

In [ ]:
df_wild_unique_english = df_wild_unique[df_wild_unique.Cluster_ID.isin(df_cluster_refied[df_cluster_refied.Tweet_Lang == 'en'].groupby('Cluster_ID').count().reset_index().Cluster_ID.tolist())]

In [ ]:
df_wild_unique_english = df_wild_unique_english.reset_index(drop = True)

In [ ]:
docs_wild_unique_english = df_wild_unique_english.Tweet_Text.tolist()

In [ ]:
hdbscan_model5 = HDBSCAN(min_cluster_size=2, metric='euclidean', 
                        cluster_selection_method='eom', prediction_data=True, min_samples=2)
representation_model5 = KeyBERTInspired()
topic_model_wild_english = BERTopic(representation_model=representation_model5, language="multilingual", 
                                    hdbscan_model=hdbscan_model5)
topics_wild_unique_english, probs_wild_unique_english = topic_model_wild_english.fit_transform(docs_wild_unique_english)
new_topics_wild_unique_english = topic_model_wild_english.reduce_outliers(docs_wild_unique_english, topics_wild_unique_english)
topic_model_wild_english.save("/disk/mnemo/users/randomuser/SBERT All Embedding/Group_Topic/Topic_Wild_Unique_English", serialization="pickle")


In [ ]:
similar_topics, similarity =topic_model_wild_unique.find_topics("oxygen", top_n=20)

In [ ]:
topic_model_wild_unique.get_topic(similar_topics[0])

In [ ]:
topic_model_wild_unique.get_topic_info()[topic_model_wild_unique.
                                         get_topic_info().
                                         Topic == similar_topics[1]]

In [ ]:
similar_topics

In [ ]:
similarity

In [ ]:
df_wild_unique = df_wild_unique.reset_index(drop = True)

In [ ]:
df_wild_unique.to_csv('Campaing_Topic_Map.csv', index = False)

In [ ]:
topic_model_wild_unique.get_topic_info()[10:20]

In [ ]:
df_wild_unique.loc[df_wild_unique.Actual_Topic == 6].iloc[100:130]

In [ ]:
df_elect = df_wild_unique.loc[df_wild_unique.Actual_Topic == 10]

In [ ]:
df_elect

In [ ]:
df_uae = df_wild_unique[df_wild_unique.apply(lambda row: row.astype(str).str.contains('UAE').any(), axis=1)]

In [ ]:
df_uae.iloc[40:]

In [ ]:
df_cluster_refied.loc[df_cluster_refied.Cluster_ID == 'Wild_67434'][['Retweet_Count', 'Favorite_Count' ]].sum().sum()

In [ ]:
df_account

In [ ]:
df_cluster_refied.loc[df_cluster_refied.Cluster_ID == 'Wild_59666']['Tweet_Text']

In [ ]:
df_wild_unique[['Cluster_ID', 'Reduce_Topic', 'Actual_Topic']]

In [ ]:
pd.merge(df_cluster_refied, df_wild_unique[['Cluster_ID', 'Reduce_Topic', 'Actual_Topic']]).drop(columns = {'Unnamed: 0.1', 'Unnamed: 0'}).to_csv('All_Tweet_With_Topic.csv', index = False)

In [ ]:
df_wild_unique[df_wild_unique.Cluster_ID.isin(df_cluster_particate.Cluster_ID)].groupby('Reduce_Topic').count()

In [ ]:
top_community = [0,1,8,6, 15, 4, 13, 14, 5, 18, 17, 19, 29, 42]

In [ ]:
df_cluster_particate[df_cluster_particate.Community_ID.isin(top_community)].Community_ID

In [ ]:
df_wild_unique[df_wild_unique.Cluster_ID.isin(df_cluster_particate[df_cluster_particate.Community_ID.isin(top_community)].Cluster_ID)].groupby('Reduce_Topic').count()

In [ ]:
df_wild_unique.groupby('Reduce_Topic').count()

In [ ]:
df_wild_unique[df_wild_unique.Tweet_Lang == 'hi']

In [ ]:
df_cluster_refied[['Cluster_ID', 'Tweet_Lang']].drop_duplicates('Cluster_ID')

In [ ]:
df_wild_unique = pd.merge(df_wild_unique, df_cluster_refied[['Cluster_ID', 'Tweet_Lang']].drop_duplicates('Cluster_ID'))

In [ ]:
df_outlier_hindi = df_wild_unique[(df_wild_unique.Actual_Topic == -1) & (df_wild_unique.Tweet_Lang == 'hi')]

In [ ]:
df_cluster_refied[df_cluster_refied.Cluster_ID.isin(df_outlier_hindi.Cluster_ID.tolist())].groupby('Cluster_ID').count().reset_index().describe()

In [ ]:
outlizer_camp_size = df_cluster_refied[df_cluster_refied.Cluster_ID.isin(df_outlier_hindi.Cluster_ID.tolist())].groupby('Cluster_ID').count().reset_index()

In [ ]:
outlizer_camp_size[outlizer_camp_size.Tweet_ID >= 5]

In [ ]:
df_outlier_hindi

# All English Campaign Topic

In [ ]:
df_wild_unique.to_csv('Tweet_For_Translate.csv', index = False)

In [ ]:
df_wild_unique

In [ ]:
hdbscan_model_all_english = HDBSCAN(min_cluster_size=2, metric='euclidean', 
                        cluster_selection_method='eom', prediction_data=True, min_samples=2)
representation_model_all_english = KeyBERTInspired()

In [ ]:
df_cluster_all_english = pd.read_csv('Updated_Translated_Campaign_for_Narrative.csv', lineterminator = '\n')

In [ ]:
df_cluster_all_english.columns

In [ ]:
df_cluster_all_english['Refined_Text_English'] = df_cluster_all_english['Translated_Tweets'].apply(remove_twitter_mentions)
df_cluster_all_english['Refined_Text_English'] = df_cluster_all_english['Refined_Text_English'].apply(remove_links)
df_cluster_all_english['Refined_Text_English'] = df_cluster_all_english['Refined_Text_English'].str.replace('\n', ' ')
df_cluster_all_english['Refined_Text_English'] = df_cluster_all_english['Refined_Text_English'].str.strip()

In [ ]:
doc_all_English = df_cluster_all_english['Refined_Text_English'].tolist()

In [ ]:
doc_all_English

In [ ]:
topic_model_all_english = BERTopic(representation_model=representation_model_all_english, hdbscan_model=hdbscan_model_all_english)
topics_all_english, probs_all_english = topic_model_all_english.fit_transform(doc_all_English[0:50])

In [ ]:
topic_model_all_english.get_topic_info()

In [ ]:
from bertopic.representation import KeyBERTInspired
from bertopic.representation import PartOfSpeech
from bertopic.representation import MaximalMarginalRelevance
from sklearn.datasets import fetch_20newsgroups
from bertopic.representation import ZeroShotClassification


main_representation = KeyBERTInspired()

candidate_topics = ['politics','religion', 'china', 'pakistan', 'bjp', 'congress']
aspect_model1 = PartOfSpeech("en_core_web_sm")
aspect_model2 = [KeyBERTInspired(top_n_words=30), MaximalMarginalRelevance(diversity=.5)]

representation_model = {
   "Main": main_representation,
   "Aspect1":  aspect_model1,
   "Aspect2":  aspect_model2 
}
topic_model_all_english = BERTopic(representation_model=representation_model, hdbscan_model=hdbscan_model_all_english)
topics_all_english, probs_all_english = topic_model_all_english.fit_transform(doc_all_English)
topic_model_all_english.save("/disk/mnemo/users/randomuser/SBERT All Embedding/Group_Topic/Topic_All_English_Multi_Aspect", serialization="pickle")
new_topics_all_english_up = topic_model_all_english.reduce_outliers(doc_all_English, topics_all_english)
topic_model_all_english.save("/disk/mnemo/users/randomuser/SBERT All Embedding/Group_Topic/Topic_All_English_Multi_Aspect_Reduce_Outlier", serialization="pickle")


In [ ]:
new_topics_all_english_up

In [ ]:
df_cluster_all_english_small = df_cluster_all_english[['Cluster_ID', 'Tweet_ID']]
df_cluster_all_english_small['Topic_All_English_Actual'] = topics_all_english
df_cluster_all_english_small['Topic_All_English_Reduce'] = new_topics_all_english_up

In [ ]:
df_cluster_all_english_small['Topic_All_English_Actual'] = topics_all_english
df_cluster_all_english_small['Topic_All_English_Reduce'] = new_topics_all_english_up

In [ ]:
topic_model_all_english.get_topic_info().to_csv('/disk/mnemo/users/randomuser/SBERT All Embedding/Topic_Model_Results/Topic_All_English.csv', index = False)
df_cluster_all_english_small.to_csv('/disk/mnemo/users/randomuser/SBERT All Embedding/Topic_Model_Results/Campaign_All_English.csv', index = False)

In [ ]:
df_cluster_all_english_small.to_csv('/disk/mnemo/users/randomuser/SBERT All Embedding/Topic_Model_Results/Campaign_All_English.csv', index = False)

In [ ]:
topic_model_all_english.save("/disk/mnemo/users/randomuser/SBERT All Embedding/Group_Topic/Topic_All_English_Multi_Aspect", serialization="pickle")
new_topics_all_english_up = topic_model_all_english.reduce_outliers(doc_all_English, topics_all_english)
topic_model_all_english.save("/disk/mnemo/users/randomuser/SBERT All Embedding/Group_Topic/Topic_All_English_Multi_Aspect_Reduce_Outlier", serialization="pickle")

In [ ]:
topic_model_all_english.get_topic_info()

In [ ]:
similar_topics, similarity =topic_model_all_english.find_topics("china", top_n=20)

In [ ]:
topic_model_all_english_single = BERTopic(representation_model=representation_model_all_english, hdbscan_model=hdbscan_model_all_english)
topics_all_english_single, probs_all_english_single = topic_model_all_english_single.fit_transform(doc_all_English)

topic_model_all_english_single.save("/disk/mnemo/users/randomuser/SBERT All Embedding/Group_Topic/Topic_All_English", serialization="pickle")
new_topics_all_english_single_up = topic_model_all_english_single.reduce_outliers(doc_all_English, topics_all_english_single)

topic_model_all_english_single.save("/disk/mnemo/users/randomuser/SBERT All Embedding/Group_Topic/Topic_All_English_Reduce_Outlier", serialization="pickle")

In [ ]:
df_cluster_all_english_small = df_cluster_all_english[['Cluster_ID', 'Tweet_ID']]
df_cluster_all_english_small['Topic_All_English_Actual'] = topics_all_english_single
df_cluster_all_english_small['Topic_All_English_Reduce'] = new_topics_all_english_single_up


In [ ]:
df_cluster_all_english_small

In [ ]:
topic_model_all_english_single.get_topic_info().to_csv('/disk/mnemo/users/randomuser/SBERT All Embedding/Topic_Model_Results/Topic_All_English_Single_Aspect.csv', index = False)
df_cluster_all_english_small.to_csv('/disk/mnemo/users/randomuser/SBERT All Embedding/Topic_Model_Results/Campaign_All_English_Single_Aspect.csv', index = False)

In [ ]:
topic_model_all_english2.get_topic_info()

In [ ]:
topic_model_all_english2 = BERTopic(representation_model=representation_model, hdbscan_model=hdbscan_model_all_english)
topics_all_english2, probs_all_english2 = topic_model_all_english2.fit_transform(doc_all_English)
topic_model_all_english2.save("/disk/mnemo/users/randomuser/SBERT All Embedding/Group_Topic/Topic_All_English_Multi_Aspect2", serialization="pickle")
new_topics_all_english_up2 = topic_model_all_english2.reduce_outliers(doc_all_English, topics_all_english2)
topic_model_all_english2.save("/disk/mnemo/users/randomuser/SBERT All Embedding/Group_Topic/Topic_All_English_Multi_Aspect_Reduce_Outlier2", serialization="pickle")


In [ ]:
df_cluster_all_english_small = df_cluster_all_english[['Cluster_ID', 'Tweet_ID']]
df_cluster_all_english_small['Topic_All_English_Actual'] = topics_all_english2
df_cluster_all_english_small['Topic_All_English_Reduce'] = new_topics_all_english_up2

In [ ]:
topic_model_all_english2.get_topic_info().to_csv('/disk/mnemo/users/randomuser/SBERT All Embedding/Topic_Model_Results/Topic_All_English2.csv', index = False)
df_cluster_all_english_small.to_csv('/disk/mnemo/users/randomuser/SBERT All Embedding/Topic_Model_Results/Campaign_All_English2.csv', index = False)

In [ ]:
hdbscan_model_all_english_up = HDBSCAN(min_cluster_size=5, metric='euclidean', 
                        cluster_selection_method='eom', prediction_data=True, min_samples=2)
representation_model_all_english = KeyBERTInspired()



topic_model_all_english3 = BERTopic(representation_model=representation_model, hdbscan_model=hdbscan_model_all_english_up)
topics_all_english3, probs_all_english3 = topic_model_all_english3.fit_transform(doc_all_English)
topic_model_all_english3.save("/disk/mnemo/users/randomuser/SBERT All Embedding/Group_Topic/Topic_All_English_Multi_Aspect3", serialization="pickle")


In [ ]:
new_topics_all_english_up3

In [ ]:
new_topics_all_english_up3 = topic_model_all_english3.reduce_outliers(doc_all_English, topics_all_english3)
topic_model_all_english3.save("/disk/mnemo/users/randomuser/SBERT All Embedding/Group_Topic/Topic_All_English_Multi_Aspect_Reduce_Outlier3", serialization="pickle")


In [ ]:
df_cluster_all_english_small = df_cluster_all_english[['Cluster_ID', 'Tweet_ID']]
df_cluster_all_english_small['Topic_All_English_Actual'] = topics_all_english3
df_cluster_all_english_small['Topic_All_English_Reduce'] = new_topics_all_english_up3

In [ ]:
df_cluster_all_english_small

In [ ]:
topic_model_all_english3.get_topic_info().to_csv('/disk/mnemo/users/randomuser/SBERT All Embedding/Topic_Model_Results/Topic_All_English3.csv', index = False)
df_cluster_all_english_small.to_csv('/disk/mnemo/users/randomuser/SBERT All Embedding/Topic_Model_Results/Campaign_All_English3.csv', index = False)

In [ ]:
topic_model_all_english_single.get_topic_info()

In [ ]:
topic_model_all_english_single2 = BERTopic(representation_model=representation_model_all_english, hdbscan_model=hdbscan_model_all_english)
topics_all_english_single2, probs_all_english_single2 = topic_model_all_english_single2.fit_transform(doc_all_English)

topic_model_all_english_single2.save("/disk/mnemo/users/randomuser/SBERT All Embedding/Group_Topic/Topic_All_English2", serialization="pickle")
new_topics_all_english_single_up2 = topic_model_all_english_single2.reduce_outliers(doc_all_English, topics_all_english_single2)

topic_model_all_english_single2.save("/disk/mnemo/users/randomuser/SBERT All Embedding/Group_Topic/Topic_All_English_Reduce_Outlier2", serialization="pickle")

In [ ]:
df_cluster_all_english_small = df_cluster_all_english[['Cluster_ID', 'Tweet_ID']]
df_cluster_all_english_small['Topic_All_English_Actual'] = topics_all_english_single2
df_cluster_all_english_small['Topic_All_English_Reduce'] = new_topics_all_english_single_up2
topic_model_all_english_single2.get_topic_info().to_csv('/disk/mnemo/users/randomuser/SBERT All Embedding/Topic_Model_Results/Topic_All_English_Single_Aspect2.csv', index = False)
df_cluster_all_english_small.to_csv('/disk/mnemo/users/randomuser/SBERT All Embedding/Topic_Model_Results/Campaign_All_English_Single_Aspect2.csv', index = False)

In [ ]:
df_cluster_all_english_small

In [ ]:
topic_model_all_english_single2.get_topic_info()

In [ ]:
topic_model_all_english_single2.get_topic_info().to_csv('/disk/mnemo/users/randomuser/SBERT All Embedding/Topic_Model_Results/Topic_All_English_Single_Aspect2.csv', index = False)
df_cluster_all_english_small.to_csv('/disk/mnemo/users/randomuser/SBERT All Embedding/Topic_Model_Results/Campaign_All_English_Single_Aspect2.csv', index = False)

In [ ]:
topic_model_all_english_single2.get_topic_info().iloc[0:30]

In [ ]:
topic_model_all_english_single2.find_topics("china", top_n=20)

In [ ]:
topic_model_all_english_single2.get_topic_info()[topic_model_all_english_single2.get_topic_info().Topic == 1997]

In [ ]:
topic_model_all_english_single2.get_topic_info().iloc[0:20]

In [ ]:
topic_model_all_english.get_topic_info()[topic_model_all_english2.get_topic_info().Count >= 50]

In [ ]:
topic_model_all_english2.get_topic_info().iloc[0:20]

In [ ]:
df_cluster_all_english_small = df_cluster_all_english[['Cluster_ID', 'Tweet_ID', 'Refined_Text_English']]

In [ ]:
df_cluster_all_english_small.sample(20)

In [ ]:
df_cluster_all_english_small.to_csv('English_Tweet_Refied.csv', index = False)